In [1]:
import boto3
import json
import base64
import io
from docx import Document
from PIL import Image
from IPython.display import Video
from moviepy.editor import ImageClip, TextClip, CompositeVideoClip, concatenate_videoclips

In [2]:
bedrock = boto3.client('bedrock-runtime', region_name='us-east-1')

docx_file_path = 'holiday_blog.docx'
text = ""

doc = Document(docx_file_path)
for paragraph in doc.paragraphs:
    text += paragraph.text + "\n"

model_id = 'ai21.j2-ultra-v1'

response = bedrock.invoke_model(
    modelId=model_id,
    body=json.dumps({
        "prompt": f"Summarize the following text: {text}",
        "maxTokens": 100
    }),
    contentType="application/json"
)

response_body = response['body'].read().decode('utf-8')
response_json = json.loads(response_body)

summary_result = response_json['completions'][0]['data']
summary = summary_result['text']
print("Summary:", summary)

Summary:


In [3]:
text_segments = summary.strip().split('. ')

model_id = 'stability.stable-diffusion-xl-v1'

generated_images = []
for i, segment in enumerate(text_segments):
    response = bedrock.invoke_model(
        modelId=model_id,
        body=json.dumps({
            "text_prompts": [{"text": segment}],
            "cfg_scale": 7,
            "steps": 50,
            "seed": 0
        }),
        contentType="application/json"
    )
    
    response_body = response['body'].read().decode('utf-8')
    response_json = json.loads(response_body)
    
    image_data = response_json['artifacts'][0]['base64'] 
    
    image_bytes = io.BytesIO(base64.b64decode(image_data))
    img = Image.open(image_bytes)
    
    img.save(f'generated_image_{i}.png')
    generated_images.append(f'generated_image_{i}.png')

print("Generated images:", generated_images)
folder_path = "/Users/ima/Documents/project/aipoint_tutorials/image_*.png" 

Generated images:


In [6]:
import glob
from PIL import Image
import numpy as np
from moviepy.editor import ImageClip, concatenate_videoclips, AudioFileClip

# Adjust this to the full path of the folder containing your images
folder_path = "/Users/ima/Documents/project/aipoint_tutorials/image_*.png" 
audio_path = "/Users/ima/Documents/project/aipoint_tutorials/audio.mp3"  # Path to your audio file

# Read and collect multiple images
images = []
file_paths = glob.glob(folder_path)

# Check if file paths were found
if not file_paths:
    print("No images found. Check your folder path and file pattern.")
else:
    print(f"Found {len(file_paths)} images.")

for file_path in file_paths:
    image = Image.open(file_path)
    images.append(np.array(image))  # Convert PIL image to NumPy array

# Create clips if images are loaded
if images:
    clips = [ImageClip(img).set_duration(6) for img in images]
    video = concatenate_videoclips(clips)

    # Load the audio file and set it to the video
    audio = AudioFileClip(audio_path)
    video = video.set_audio(audio)

    # Save the final video with audio
    video.write_videofile("output_video_with_audio.mp4", fps=24)
else:
    print("No images to process.")


Found 5 images.
Moviepy - Building video output_video_with_audio.mp4.
MoviePy - Writing audio in output_video_with_audioTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_video_with_audio.mp4



Moviepy - Done !
Moviepy - video ready output_video_with_audio.mp4


In [7]:
Video("output_video_with_audio.mp4", embed=True)


NameError: name 'Video' is not defined